# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

df.columns = (df.columns
                .str.strip()
                .str.lower()
                .str.replace(r"\s+", "_", regex=True))

if "response" in df.columns:
    df["response"] = df["response"].astype(str).str.strip().str.title()

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [ ]:
df["total_claim_amount"] = pd.to_numeric(df["total_claim_amount"], errors="coerce")

low_claim_yes = df[(df["total_claim_amount"] < 1000) & (df["response"] == "Yes")].copy()
low_claim_yes.head()

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [ ]:
for col in ["monthly_premium_auto", "customer_lifetime_value", "total_claim_amount"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

yes_df = df[df["response"] == "Yes"].copy()

avg_premium_clv = (yes_df
                   .groupby(["policy_type", "gender"])
                   [["monthly_premium_auto", "customer_lifetime_value"]]
                   .mean()
                   .round(2))

avg_claim = (yes_df
             .groupby(["policy_type", "gender"])
             [["total_claim_amount"]]
             .mean()
             .round(2))

Comparetables = avg_premium_clv.join(avg_claim)
print(Comparetables)

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [ ]:
if "customer" in df.columns:
    customers_by_state = df.groupby("state")["customer"].nunique().rename("num_customers")
else:
    customers_by_state = df.groupby("state").size().rename("num_customers")

top_states_over_500_customers = customers_by_state[customers_by_state > 500].sort_values(ascending=False)
print(top_states_over_500_customers)

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
clv_stats = (df
             .groupby(["education", "gender"])["customer_lifetime_value"]
             .agg(["min", "median", "max"])
             .round(2)
            )
print(clv_stats)

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

df.columns = (df.columns
                .str.strip()
                .str.lower()
                .str.replace(r"\s+", "_", regex=True))

df["effective_to_date"] = pd.to_datetime(df["effective_to_date"], errors="coerce")
df["month"] = df["effective_to_date"].dt.month_name()

In [ ]:
policies_by_state_month = (
    df.pivot_table(
        index="state",
        columns="month",
        values="policy",
        aggfunc="count"
    )
    .fillna(0)
    .astype(int)
)

print(policies_by_state_month.head())

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [ ]:
top3_states = policies_by_state_month.sum(axis=1).sort_values(ascending=False).head(3)
print("Top 3 states with highest policy sales:")
print(top3_states)

In [ ]:
top3_policies_by_month = policies_by_state_month.loc[top3_states.index]

print("\nPolicies sold by month for the top 3 states:")
print(top3_policies_by_month)

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here